# Data sanity check process

In this notebook, we will extract a dataset and the content of a `data_format_ref` file

we will run a data sanity check on this dataset based on the content of`data_format_ref`

In [1]:
import pandas as pd

import fedbiomed.common.data_tool.utils as utils
import fedbiomed.common.data_tool.multi_view_dataframe as multiview

from fedbiomed.common.data_tool.warning_logger import WarningReportLogger
from fedbiomed.common.data_tool.pre_processing_checker import PreProcessingChecker

In [2]:
# loading dataset

multi_dataset_to_check = utils.load_tabular_datasets(r'demo_poc_data_wrapper/data/test7')

directory found


In [3]:
# loading `data_fromat_ref`

multi_format_file_ref = utils.load_format_file_ref('multi_format_file')

In [28]:
multi_format_file_ref

{'file1': {'e': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  '1': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': False},
  '2': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': True},
  'time': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': 'object',
   'is_missing_values': False},
  'pressure': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'e.1': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'gender': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': False},
  'blood type': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': True},
  '

In [ ]:
we wil extract

In [4]:
# select only features in dataset that will be checked
pre_parsed_dataset_to_check = multiview.select_data_from_format_file_ref(multi_dataset_to_check,
                                                                         multi_format_file_ref)

the dataset loaded is a dataset composed from multiple CSV files. To convert those dataset into a single one, we will do a join operation based on a primary key

let's  find this primary key:

In [5]:
primary_key = multiview.search_primary_key(multi_format_file_ref)

found primary key pkey
found primary key pkey
found primary key pkey


In [6]:
# extract views names
views_names = list(multi_format_file_ref.keys())

# rename columns names before join operation


pre_parsed_dataset_to_check, new_views_name = multiview.rename_variables_before_joining(pre_parsed_dataset_to_check, views_names,
                                                             primary_key)

In [7]:
# perform a join operation

# jointure operation (takesplace only if primary key has been specfied in foramt_file)
df_joined = multiview.join_multi_view_dataset(pre_parsed_dataset_to_check, primary_key, False)

In [8]:
df_joined

,discrete,city,pkey,e,file1.1,2,file1.time,pressure,e.1,gender,blood type,file2.1,file2.time,pH
0,64.0,Lille,qpqorfhylu gmfjy bdj,16,False,False,2018-01-03 04:00:00,0.986670,98,WOMAN,A,False,2018-01-02 06:00:00,NaN
1,26.0,Lille,kkmjozalfyirgsire ui,96,True,True,2018-01-02 04:00:00,0.996889,35,MAN,AB,True,2018-01-01 00:00:00,0.023107
2,61.0,Paris,ezfasuuycdda foisjte,8,True,True,2018-01-01 09:00:00,0.777026,65,MAN,A,False,2018-01-02 10:00:00,0.587685
3,29.0,Paris,faxiqkt xggzmwzoidbg,6,True,False,2018-01-04 20:00:00,0.877527,81,MAN,AB,True,2018-01-03 12:00:00,0.894073
4,99.0,Lille,znwhlj rwzdutnagwasy,79,True,True,2018-01-04 09:00:00,0.447389,88,WOMAN,O,True,2018-01-01 10:00:00,0.026831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.0,Paris,zeqhcikzdodus jn qjf,62,True,True,2018-01-02 13:00:00,0.953184,53,MAN,AB,False,2018-01-02 05:00:00,0.788560
96,98.0,Marseille,iicthcvfmkajbvr gzir,49,False,True,2018-01-02 21:00:00,0.442283,35,MAN,NaN,True,2018-01-05 02:00:00,0.402979
97,21.0,Lille,ztjakcsk bhjoksdz lm,14,False,False,2018-01-02 06:00:00,0.988543,67,MAN,B,False,2018-01-01 12:00:00,NaN
98,42.0,Marseille,sabunaa opt vpulnxj,10,True,False,2018-01-02 01:00:00,0.059791,48,MAN,B,True,2018-01-02 09:00:00,0.651801


In [10]:
# convert joined dataset into multi view dataset
new_feature_name = { v: list(pre_parsed_dataset_to_check[v].columns) for v in views_names}
new_feature_name


multiview.create_multi_view_dataframe_from_dataframe(df_joined,new_feature_name, primary_key=primary_key)

length ['e', 'file1.1', '2', 'file1.time', 'pressure', 'e.1', 'gender', 'blood type', 'discrete', 'city', 'file2.1', 'file2.time', 'pH'] ['file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'file1', 'contatct', 'contatct', 'file2', 'file2', 'file2']


views        file1                                                           \
feature_name     e file1.1      2           file1.time  pressure e.1 gender   
0               16   False  False  2018-01-03 04:00:00   0.98667  98  WOMAN   
1               96    True   True  2018-01-02 04:00:00  0.996889  35    MAN   
2                8    True   True  2018-01-01 09:00:00  0.777026  65    MAN   
3                6    True  False  2018-01-04 20:00:00  0.877527  81    MAN   
4               79    True   True  2018-01-04 09:00:00  0.447389  88  WOMAN   
..             ...     ...    ...                  ...       ...  ..    ...   
95              62    True   True  2018-01-02 13:00:00  0.953184  53    MAN   
96              49   False   True  2018-01-02 21:00:00  0.442283  35    MAN   
97              14   False  False  2018-01-02 06:00:00  0.988543  67    MAN   
98              10    True  False  2018-01-02 01:00:00  0.059791  48    MAN   
99              89    True   True  2018-01-03 22:00:00  0.939352  13    MAN   

views                   contatct              file2                       \
feature_name blood type discrete       city file2.1           file2.time   
0                     A     64.0      Lille   False  2018-01-02 06:00:00   
1                    AB     26.0      Lille    True  2018-01-01 00:00:00   
2                     A     61.0      Paris   False  2018-01-02 10:00:00   
3                    AB     29.0      Paris    True  2018-01-03 12:00:00   
4                     O     99.0      Lille    True  2018-01-01 10:00:00   
..                  ...      ...        ...     ...                  ...   
95                   AB      9.0      Paris   False  2018-01-02 05:00:00   
96                  NaN     98.0  Marseille    True  2018-01-05 02:00:00   
97                    B     21.0      Lille   False  2018-01-01 12:00:00   
98                    B     42.0  Marseille    True  2018-01-02 09:00:00   
99                    B      3.0      Paris   False  2018-01-03 02:00:00   

views                            primary_key  
feature_name        pH                  pkey  
0                  NaN  qpqorfhylu gmfjy bdj  
1             0.023107  kkmjozalfyirgsire ui  
2             0.587685  ezfasuuycdda foisjte  
3             0.894073  faxiqkt xggzmwzoidbg  
4             0.026831  znwhlj rwzdutnagwasy  
..                 ...                   ...  
95             0.78856  zeqhcikzdodus jn qjf  
96            0.402979  iicthcvfmkajbvr gzir  
97                 NaN  ztjakcsk bhjoksdz lm  
98            0.651801   sabunaa opt vpulnxj  
99            0.751969  qmbexyexvgromrm admu  

[100 rows x 14 columns]

## Perform a data sanity check

In [9]:
# define some global varibales

MIN_NB_SAMPLES = 30  # TODO: specify it in data formt file (clinician should define it)

In [23]:
warning_report = WarningReportLogger(disclosure=3)

checker = PreProcessingChecker(multi_format_file_ref, df_joined,
                               'multiview_dataFormat_file' ,
                               warning_report,
                              min_nb_samples=30)

checker.update_views_features_name(new_views_name)
checker.check_all()



features names updated
N_SAMPLES_BELOW_THRESHOLD
file1 e
file1 e
INCORRECT_FORMAT_FILE
file1 e
DATA_TYPE_MISMATCH
INCORRECT_DATA_TYPE
file1 e
MISSING_DATA_ALLOWED
file1 e
OUTLIER_DETECTION_LOWER_BOUND
file1 e
OUTLIER_DETECTION_UPPER_BOUND
file1 e
INCORRECT_VALUES_CATEGORICAL_DATA
file1 1
file1 1
file1 1
file1 1
file1 1
file1 1
file1 1
file1 2
file1 2
file1 2
file1 2
file1 2
file1 2
file1 2
file1 time
file1 time
file1 time
file1 time
file1 time
file1 time
file1 time
file1 time
INCORRECT_DATETIME_DATA
file1 pressure
file1 pressure
file1 pressure
file1 pressure
file1 pressure
file1 pressure
file1 pressure
file1 e.1
file1 e.1
file1 e.1
file1 e.1
file1 e.1
file1 e.1
file1 e.1
file1 gender
file1 gender
file1 gender
file1 gender
file1 gender
file1 gender
file1 gender
file1 blood type
file1 blood type
file1 blood type
file1 blood type
file1 blood type
file1 blood type
file1 blood type
file1 pkey
file1 pkey
file1 pkey
file1 pkey
file1 pkey
file1 pkey
file1 pkey
contatct discrete
contatct discre

## get report

In [24]:
report = checker.get_warning_logger()

number of warnings: 1
Number of error: 1


In [25]:
report

{'N_SAMPLES_BELOW_THRESHOLD': [{'view': 'file1',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'contatct',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'file2', 'success': True, 'feature': 'ALL', 'msg': 'Test passed'}],
 'INCORRECT_FORMAT_FILE': [{'view': '',
   'success': True,
   'feature': 'e',
   'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': '1', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': '2', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'time', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'pressure', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'e.1', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'gender', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'blood type', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'pkey', 'msg': 'Test passed'},
  {'view': ''

In [26]:

report2 = {}

for k in report.keys():
    report2[k] = pd.DataFrame(report[k])

report2.pop('N_SAMPLES_BELOW_THRESHOLD')
report2.pop('INCORRECT_DATETIME_DATA')
multiview.create_multi_view_dataframe_from_dictionary(report2, ['test name', 'index'])

test name INCORRECT_FORMAT_FILE                                   \
index                      view success     feature          msg   
0                                  True           e  Test passed   
1                                  True           1  Test passed   
2                                  True           2  Test passed   
3                                  True        time  Test passed   
4                                  True    pressure  Test passed   
5                                  True         e.1  Test passed   
6                                  True      gender  Test passed   
7                                  True  blood type  Test passed   
8                                  True        pkey  Test passed   
9                                  True    discrete  Test passed   
10                                 True        city  Test passed   
11                                 True        pkey  Test passed   
12                                 True           1  Test passed   
13                                 True        time  Test passed   
14                                 True          pH  Test passed   
15                                 True        pkey  Test passed   

test name DATA_TYPE_MISMATCH                                   \
index                   view success     feature          msg   
0                               True           e  Test passed   
1                               True           1  Test passed   
2                               True           2  Test passed   
3                               True        time  Test passed   
4                               True    pressure  Test passed   
5                               True         e.1  Test passed   
6                               True      gender  Test passed   
7                               True  blood type  Test passed   
8                               True        pkey  Test passed   
9                               True    discrete  Test passed   
10                              True        city  Test passed   
11                              True        pkey  Test passed   
12                              True           1  Test passed   
13                              True        time  Test passed   
14                              True          pH  Test passed   
15                              True        pkey  Test passed   

test name INCORRECT_DATA_TYPE          ... OUTLIER_DETECTION_LOWER_BOUND  \
index                    view success  ...                       feature   
0                                True  ...                             e   
1                                True  ...                             1   
2                                True  ...                             2   
3                                True  ...                          time   
4                                True  ...                      pressure   
5                                True  ...                           e.1   
6                                True  ...                        gender   
7                                True  ...                    blood type   
8                                True  ...                          pkey   
9                                True  ...                      discrete   
10                               True  ...                          city   
11                               True  ...                          pkey   
12                               True  ...                             1   
13                               True  ...                          time   
14                               True  ...                            pH   
15                               True  ...                          pkey   

test name               OUTLIER_DETECTION_UPPER_BOUND                      \
index               msg                          view success     feature   
0          Test skipped                                  None           e   
1   

In [13]:
report

{'N_SAMPLES_BELOW_THRESHOLD': [{'view': 'file1',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'contatct',
   'success': True,
   'feature': 'ALL',
   'msg': 'Test passed'},
  {'view': 'file2', 'success': True, 'feature': 'ALL', 'msg': 'Test passed'}],
 'INCORRECT_FORMAT_FILE': [{'view': '',
   'success': True,
   'feature': 'e',
   'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': '1', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': '2', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'time', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'pressure', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'e.1', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'gender', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'blood type', 'msg': 'Test passed'},
  {'view': '', 'success': True, 'feature': 'pkey', 'msg': 'Test passed'},
  {'view': ''

# raise collected error

This will raises all collected error found in the warning logger

In [27]:
checker.get_warning_logger.raise_exception()

DataSanityCheckException: MissingDataException: Variable pH must not have missing data, but some were found